In [1]:
from queue import PriorityQueue
# Heuristic function to estimate the cost of reaching the goal state from the current state
def heuristic(state):
    m_left, c_left, b_pos, m_right, c_right = state
    return (m_left + c_left - 2) // 2 + (m_right + c_right - 2) // 2
# Function to check if a state is valid
def is_valid(state):
    m_left, c_left, b_pos, m_right, c_right = state
    if m_left < 0 or c_left < 0 or m_right < 0 or c_right < 0:
        return False
    if m_left > 3 or c_left > 3 or m_right > 3 or c_right > 3:
        return False
    if (c_left > m_left > 0) or (c_right > m_right > 0):
        return False
    return True
# Function to generate the next valid states from the current state
def next_states(state):
    m_left, c_left, b_pos, m_right, c_right = state
    if b_pos == 'left':
        moves = [(2, 0), (0, 2), (1, 1), (1, 0), (0, 1)]
        next_states = [(m_left-m, c_left-c, 'right', m_right+m, c_right+c) for m, c in moves]
    else:
        moves = [(-2, 0), (0, -2), (-1, -1), (-1, 0), (0, -1)]
        next_states = [(m_left+m, c_left+c, 'left', m_right-m, c_right-c) for m, c in moves]
    return [state for state in next_states if is_valid(state)]
# A* Search algorithm
def astar(start_state):
    frontier = PriorityQueue()  # Priority queue to explore states based on the sum of cost and heuristic value
    frontier.put((0, [start_state]))  # Insert the initial state with cost 0
    explored = set()    
    while not frontier.empty():
        cost, path = frontier.get()  # Get the state with lowest cost from the priority queue
        current_state = path[-1]        
        if current_state == (0, 0, 'right', 3, 3):
            return path        
        if current_state not in explored:
            explored.add(current_state)
            for next_state in next_states(current_state):
                new_cost = len(path) - 1  # Uniform cost for each move
                new_path = path + [next_state]
                frontier.put((new_cost + heuristic(next_state), new_path))  # Insert the new state with its priority into the priority queue
    return None
# Testing the algorithm with the initial state (3, 3, 'left', 0, 0)
start_state = (3, 3, 'left', 0, 0)
path = astar(start_state)
# Printing the path from the initial state to the goal state
if path:
    for state in path:
        print(state)
else:
    print("No solution found.")

(3, 3, 'left', 0, 0)
(2, 2, 'right', 1, 1)
(0, 2, 'left', 3, 1)
(0, 0, 'right', 3, 3)


In [10]:
import heapq
import random

# Calculate a heuristic based on how many people are left to cross
def heuristic(state):
    left_missionaries, left_cannibals, right_missionaries, right_cannibals, boat_position = state
    return left_missionaries + left_cannibals  # Total people left on the left side

# Function to generate possible moves and resulting new states
def get_possible_moves(state):
    left_missionaries, left_cannibals, right_missionaries, right_cannibals, boat_position = state
    possible_moves = []

    # Define possible transitions for moves
    transitions = [
        (1, 0),  # One missionary
        (2, 0),  # Two missionaries
        (0, 1),  # One cannibal
        (0, 2),  # Two cannibals
        (1, 1),  # One missionary and one cannibal
    ]

    # Generate the new states based on valid transitions
    for transition in transitions:
        m, c = transition  # Missionaries and cannibals to move
        if boat_position == 'left':
            new_left_m = left_missionaries - m
            new_left_c = left_cannibals - c
            new_right_m = right_missionaries + m
            new_right_c = right_cannibals + c
            new_boat_position = 'right'
        else:
            new_left_m = left_missionaries + m
            new_left_c = left_cannibals + c
            new_right_m = right_missionaries - m
            new_right_c = right_cannibals - c
            new_boat_position = 'left'

        # Ensure the move doesn't lead to cannibals outnumbering missionaries
        if (
            new_left_m >= 0
            and new_left_c >= 0
            and new_right_m >= 0
            and new_right_c >= 0
            and (new_left_m == 0 or new_left_m >= new_left_c)
            and (new_right_m == 0 or new_right_m >= new_right_c)
        ):
            possible_moves.append(
                (
                    new_left_m,
                    new_left_c,
                    new_right_m,
                    new_right_c,
                    new_boat_position,
                )
            )

    return possible_moves

# Greedy Best-First Search (GBFS) for solving the Missionaries and Cannibals problem
def gbfs(start, goal):
    # Priority queue for GBFS with the heuristic cost
    queue = []
    heapq.heappush(queue, (heuristic(start), start, []))  # (heuristic, state, path)

    visited = set()  # To track visited states to avoid cycles

    while queue:
        _, current_state, path = heapq.heappop(queue)

        if current_state == goal:
            return path  # Solution found

        if current_state not in visited:
            visited.add(current_state)

            # Explore possible moves from the current state
            for new_state in get_possible_moves(current_state):
                if new_state not in visited:
                    new_path = path + [new_state]
                    heapq.heappush(queue, (heuristic(new_state), new_state, new_path))

    return None  # No solution found

# Example usage
if __name__ == "__main__":
    # Starting state with all on the left side
    start_state = (3, 3, 0, 0, 'left')
    # Goal state with all on the right side
    goal_state = (0, 0, 3, 3, 'right')

    # Solve the Missionaries and Cannibals problem using GBFS
    solution = gbfs(start_state, goal_state)

    if solution:
        print("Greedy Best-First Search Solution:")
        for step, state in enumerate(solution):
            print(f"Step {step}: {state}")
    else:
        print("No solution found.")


Greedy Best-First Search Solution:
Step 0: (2, 2, 1, 1, 'right')
Step 1: (3, 2, 0, 1, 'left')
Step 2: (3, 0, 0, 3, 'right')
Step 3: (3, 1, 0, 2, 'left')
Step 4: (1, 1, 2, 2, 'right')
Step 5: (2, 2, 1, 1, 'left')
Step 6: (0, 2, 3, 1, 'right')
Step 7: (0, 3, 3, 0, 'left')
Step 8: (0, 1, 3, 2, 'right')
Step 9: (0, 2, 3, 1, 'left')
Step 10: (0, 0, 3, 3, 'right')
